In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

import requests
import json
from random import randint
import os
import time
import json
import pickle

In [2]:
%cd ..
%cd demo Mativ
all_anime_df = pd.read_csv(os.path.join("data", "tags_anime.csv"))
%cd ..
%cd demo Victor
all_anime_df

c:\Users\mativ\Documents\Jupyter Notebooks\Proyecto-Grupo-12
c:\Users\mativ\Documents\Jupyter Notebooks\Proyecto-Grupo-12\demo Mativ
c:\Users\mativ\Documents\Jupyter Notebooks\Proyecto-Grupo-12
c:\Users\mativ\Documents\Jupyter Notebooks\Proyecto-Grupo-12\demo Victor


,Anime,URL
0,Attack on Titan The Final Season: The Final Ch...,https://www.anime-planet.com/anime/attack-on-t...
1,Fullmetal Alchemist: Brotherhood,https://www.anime-planet.com/anime/fullmetal-a...
2,Fruits Basket the Final Season,https://www.anime-planet.com/anime/fruits-bask...
3,Demon Slayer: Kimetsu no Yaiba - Entertainment...,https://www.anime-planet.com/anime/demon-slaye...
4,Jujutsu Kaisen 2nd Season,https://www.anime-planet.com/anime/jujutsu-kai...
...,...,...
23329,The Mad Capsule Markets: Pulse,https://www.anime-planet.com/anime/the-mad-cap...
23330,Kyoufu no Hiruyasumi,https://www.anime-planet.com/anime/kyoufu-no-h...
23331,Xing Xueyuan III: Pandora Mijing,https://www.anime-planet.com/anime/xing-xueyua...
23332,Xiyou Xin Chuan,https://www.anime-planet.com/anime/xiyou-xin-c...


In [3]:
def extraer_tags(animes: pd.DataFrame, head):
    # Tiempo de espera aleatorio entre 1 y 3 segundos.
    extension = animes.copy()
    extension["Tags"] = ""
    extension["Alts"] = ""

    for index, row in animes.iterrows():
        # Tiempo de espera aleatorio entre 1 y 3 segundos.
        time.sleep(randint(1, 3))
        anime_url = row["URL"]
        anime_title = row["Anime"]

        while True:
            try:
                anime_response = requests.get(url=anime_url, headers=head)
            except requests.exceptions.RequestException as e:
                print(f"Error de internet:\n\n{e}\n\n")
                print(anime_title, anime_url)
                time.sleep(300)
                continue
            except Exception as e:
                print(f"Error desconocido:\n\n{e}\n\n")
                print(anime_title, anime_url)
                time.sleep(300)
                continue
            try:
                # Manejar Status codes.
                if anime_response.status_code != 200: # Error desconocido
                    print(anime_response.status_code)
                    print(anime_response.json())
                    print(anime_title, anime_url)
                    time.sleep(60)
                    continue

                anime_soup = bs(anime_response.content, "html.parser")
                
                # Tags del anime
                meta_tags = anime_soup.find_all("meta", {"property": "video:tag"})
                if meta_tags:
                    tags = [tag["content"] for tag in meta_tags]
                else:
                    tags = []
                
                # Titulos alternativos
                class_aka = anime_soup.find_all("h2", {"class": "aka"})
                if class_aka:
                    alt_titles = class_aka[0].contents[0]
                else:
                    alt_titles = ""

                extension.at[index, "Tags"] = tags
                extension.at[index, "Alts"] = alt_titles
                break

            # Manejar errores
            except KeyboardInterrupt: # Usuario saltado por algun error etc.
                print("anime saltado")
                print(anime_response.status_code)
                print(anime_response.json())
                print(anime_title, anime_url)
                break

    return extension

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36 Edg/94.0.992.50",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "TE": "Trailers",
}

In [ ]:
cantidad = len(all_anime_df)

for i in range(15, 21):
    conj = all_anime_df[((i-1)*cantidad)//20:(i*cantidad)//20]
    resultado = extraer_tags(conj, headers)

    path = os.path.join("data", f"tags_anime_{i}.pickle")
    resultado.to_csv(path, index=False)